In [1]:
from data_loader import PykrxDataLoader
import pandas as pd
from pykrx import stock
import numpy as np
import warnings
warnings.filterwarnings('ignore')

### 연도별 시가총액 상위 200개 기업리스트의 수익률 수집(2015~2024)

In [3]:
start_year = 2015
end_year = 2024

# 결과를 저장할 리스트 생성
records = []

for year in range(start_year, end_year + 1):
    date = f"{year}-12-31"  # 연말 기준 (휴장일일 경우 이전 영업입)
    df = stock.get_index_portfolio_deposit_file("1028", f"{year}1231", alternative = True)
    # 각 코드에 대해 연도와 함께 리스트에 저장
    for code in df:
        records.append({'연도': year, '주식코드': code})

# 리스트를 데이터프레임으로 변환
result_df = pd.DataFrame(records)

# 결과 확인
# print(result_df)

# 상위 200위 내에 들어온 기업의 빈도수
frequency = result_df['주식코드'].value_counts()
frequency_df = frequency.reset_index()
frequency_df.columns = ['주식코드', '빈도수']
# print(frequency_df)
code_df = frequency_df[['주식코드']]
# code_df['주식코드'].tolist()
# print(code_df)

# 코스피 200에 한번이라도 들어온 기업들의 연도별 수익률 수집
records = []
for code in code_df['주식코드']:
    df = stock.get_market_ohlcv_by_date(
        f"{start_year}0101", f"{end_year}1231", code, freq='y'
    )
    if not df.empty:
        df = df[['시가', '종가']].reset_index()  # 시가 포함
        df['연도'] = df['날짜'].dt.year
        # 시가 대비 종가 변화율 계산
        df['수익률'] = ((df['종가'] - df['시가']) / df['시가'])
        records.append(df.assign(주식코드=code))

# Long-form 데이터 생성
result_df = pd.concat(records)[['주식코드', '연도', '수익률']]

# print(result_df)

# 1. 무한대 값을 NaN으로 변환
result_df = result_df.replace([np.inf, -np.inf], np.nan)

# 완성된 데이터프레임을 csv로 저장
result_df.dropna(inplace=True)
result_df.to_csv("./data/rtn_data.csv", index=False)

The date you entered 20151231 seems to be a holiday. PYKRX changes the date parameter to 20151230.
The date you entered 20161231 seems to be a holiday. PYKRX changes the date parameter to 20161229.
The date you entered 20171231 seems to be a holiday. PYKRX changes the date parameter to 20171228.
The date you entered 20181231 seems to be a holiday. PYKRX changes the date parameter to 20181228.
The date you entered 20191231 seems to be a holiday. PYKRX changes the date parameter to 20191230.
The date you entered 20201231 seems to be a holiday. PYKRX changes the date parameter to 20201230.
The date you entered 20211231 seems to be a holiday. PYKRX changes the date parameter to 20211230.
The date you entered 20221231 seems to be a holiday. PYKRX changes the date parameter to 20221229.
The date you entered 20231231 seems to be a holiday. PYKRX changes the date parameter to 20231228.
The date you entered 20241231 seems to be a holiday. PYKRX changes the date parameter to 20241230.


### 재무지표

In [4]:
import pandas as pd

In [ ]:

def ts_data_loader(name):
    df = pd.read_excel(f"./data/{name}.xlsx", dtype={'거래소코드': str})
    ts_list = df.columns.unique()

    ts_df = df.copy()
    ts_df['회계년도'] = ts_df['회계년도'].str.split('/').str[0].astype(int)
    ts_df['거래소코드'] = ts_df['거래소코드'].str.strip()
    ts_df.rename(columns = {'회계년도':'연도',
                            '거래소코드':'주식코드'}, inplace=True)
    # ts_df = ts_df.drop('종가(원)', axis=1)
    ts_df = ts_df.dropna()
    # print(ts_df)

    # 중복 확인
    duplicates = ts_df.duplicated(subset=['주식코드', '연도'], keep=False)
    ts_df[duplicates]

    # 중복된 ('주식코드', '연도') 조합에서 첫 번째 값만 남김
    ts_df = ts_df.drop_duplicates(subset=['주식코드', '연도'], keep='first').sort_values(by=['주식코드','연도'])
    ts_df.to_csv(f'./data/{name}.csv', index=False, encoding="utf-8-sig")

    return ts_df

In [23]:
df_생산성 = ts_data_loader('생산성')
df_성장성 = ts_data_loader('성장성')
df_수익성 = ts_data_loader('수익성')
df_안정성 = ts_data_loader('안정성')
df_활동성 = ts_data_loader('활동성')

In [28]:
from functools import reduce
import pandas as pd

# 데이터프레임 리스트로 묶기
dfs = [df_생산성, df_성장성, df_수익성, df_안정성, df_활동성]

# 'key' 컬럼을 기준으로 내부조인
joined_df = reduce(lambda left, right: pd.merge(left, right, on=['회사명','주식코드','연도'], how='inner'), dfs)

# 결과 확인
# print(joined_df.head())

joined_df.to_csv("./data/ts_data.csv", index=False, encoding="utf-8-sig")